# **1. The problem statement**

In this kernel, I try to make predictions where the prediction task is to determine whether a person makes over 50K a year. I implement Random Forest Classification, Decision Tree and SVM with Python and Scikit-Learn. So, to answer the question, I build a Random Forest classifier, Decision Tree and SVM to predict whether a person makes over 50K a year.

# 2. Import libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
sns.set(style="whitegrid")

# 3. Import dataset

In [ ]:
data='/kaggle/input/income-classification/income_evaluation.csv'
df=pd.read_csv(data)

# 4. Exploratory data analysis

Now, I will explore the data to gain insights about the data.

In [ ]:
#print the shape
print('The shape of the dataset : ' , df.shape)

We can see that there are 32561 instances and 15 attributes in the data set.

In [ ]:
df.head()

# Rename column names

We can see that the dataset does not have proper column names. The column names contain underscore. We should give proper names to the columns. I will do it as follows:-

In [ ]:
col_names = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship',
             'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income']

df.columns = col_names

df.columns

In [ ]:
df.dtypes

# Findings

We can see that the dataset contains 9 character variables and 6 numerical variables.
income is the target variable.

In [ ]:
df.describe()

The above df.describe() command presents statistical properties in vertical form.

In [ ]:
#check for missing value

df.isnull().sum()

# Types of variables

In this section, I segregate the dataset into categorical and numerical variables.

There are a mixture of categorical and numerical variables in the dataset.

Categorical variables have data type object. Numerical variables have data type int64.

First of all, I will explore categorical variables.

In [ ]:
categorical = [var for var in df.columns if df[var].dtype=='O']

print('There are {} categorical variables\n'.format(len(categorical)))

print('The categorical variables are :\n\n', categorical)

* There are 9 categorical variables in the dataset.

* The categorical variables are given by workclass, education, marital_status, occupation, relationship, race, sex, native_country and income.

In [ ]:
df[categorical].head()

Now, we will check the frequency distribution of categorical variables.

In [ ]:
for var in categorical: 
    
    print(df[var].value_counts())

* Percentage of frequency distribution of values

# Explore income target variable

In [ ]:
# check for missing values

df['income'].isnull().sum()

**We can see that there are no missing values in the income target variable.**

In [ ]:
# view number of unique values

df['income'].nunique()

In [ ]:
# view the frequency distribution of values

df['income'].value_counts()

**The two unique values are <=50K and >50K.**

In [ ]:
# view percentage of frequency distribution of values

df['income'].value_counts()/len(df)

In [ ]:
# visualize frequency distribution of income variable

f,ax=plt.subplots(1,2,figsize=(18,8))

ax[0] = df['income'].value_counts().plot.pie(explode=[0,0],autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('Income Share')


#f, ax = plt.subplots(figsize=(6, 8))
ax[1] = sns.countplot(x="income", data=df, palette="Set1")
ax[1].set_title("Frequency distribution of income variable")

plt.show()

**We can see that males make more money than females in both the income categories.**

In [ ]:
# check number of unique labels 

df.workclass.nunique()

In [ ]:
# view frequency distribution of values

df.workclass.value_counts()

In [ ]:
# replace '?' values in workclass variable with `NaN`

df['workclass'].replace(' ?', np.NaN, inplace=True)

In [ ]:
# again check the frequency distribution of values in workclass variable

df.workclass.value_counts()

In [ ]:
f, ax = plt.subplots(figsize=(10, 6))
ax = df.workclass.value_counts().plot(kind="bar", color="green")
ax.set_title("Frequency distribution of workclass variable")
ax.set_xticklabels(df.workclass.value_counts().index, rotation=30)
plt.show()

We can see that there are lot more private workers than other category of workers.

In [ ]:
# check number of unique labels

df.occupation.nunique()

In [ ]:
# view unique labels

df.occupation.unique()

In [ ]:
# view frequency distribution of values

df.occupation.value_counts()

In [ ]:
# replace '?' values in occupation variable with `NaN`

df['occupation'].replace(' ?', np.NaN, inplace=True)

In [ ]:
# again check the frequency distribution of values

df.occupation.value_counts()

In [ ]:
# check number of unique labels

df.native_country.nunique()

In [ ]:
# view unique labels 

df.native_country.unique()

In [ ]:
# check frequency distribution of values

df.native_country.value_counts()

In [ ]:
# replace '?' values in native_country variable with `NaN`

df['native_country'].replace(' ?', np.NaN, inplace=True)
# again check the frequency distribution of values

df.native_country.value_counts()

In [ ]:
df[categorical].isnull().sum()

We can see that native_country column contains relatively large number of labels as compared to other columns. I will check for cardinality after train-test split.

# Find numerical variables

In [ ]:
numerical = [var for var in df.columns if df[var].dtype!='O']

print('There are {} numerical variables\n'.format(len(numerical)))

print('The numerical variables are :\n\n', numerical)

In [ ]:
df[numerical].head()

In [ ]:
df[numerical].isnull().sum()

We can see that there are no missing values in the numerical variables.

In [ ]:
df['age'].nunique()

In [ ]:
f, ax = plt.subplots(figsize=(10,8))
x = df['age']
ax = sns.distplot(x, bins=10, color='blue')
ax.set_title("Distribution of age variable")
plt.show()

**Explore relationship between age and income variables**

**As expected, younger people make less money as compared to senior people.**

In [ ]:
# plot correlation heatmap to find out correlations

df.corr().style.format("{:.4}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

**We can see that there is no strong correlation between variables.**

 **Declare feature vector and target variable**

In [ ]:
X=df.drop(['income'],axis=1)

y=df['income']

Split data into separate training and test set

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

**Scikit-Learn (sklearn) → Commonly used open source machine learning library**

In [ ]:
#check the shape of X_train and X_test

X_train.shape, X_test.shape

**I will do feature engineering on different variables.**

**First, I will show the categorical and numerical variables separately in the training set.**

In [ ]:
categorical = [col for col in X_train.columns if X_train[col].dtypes == 'O']

categorical

In [ ]:
numerical = [col for col in X_train.columns if X_train[col].dtypes != 'O']

numerical

In [ ]:
# print percentage of missing values in the categorical variables in training set

X_train[categorical].isnull().mean()

In [ ]:
# print categorical variables with missing data

for col in categorical:
    if X_train[col].isnull().mean()>0:
        print(col, (X_train[col].isnull().mean()))

In [ ]:
for df2 in [X_train, X_test]:
    df2['workclass'].fillna(X_train['workclass'].mode()[0], inplace=True)
    df2['occupation'].fillna(X_train['occupation'].mode()[0], inplace=True)
    df2['native_country'].fillna(X_train['native_country'].mode()[0], inplace=True)

In [ ]:
# check missing values in categorical variables in X_train

X_train[categorical].isnull().sum()

In [ ]:
# check missing values in categorical variables in X_test

X_test[categorical].isnull().sum()

**As a final check, I will check for missing values in X_train and X_test.**

In [ ]:
# check missing values in X_train

X_train.isnull().sum()

In [ ]:
# check missing values in X_test

X_test.isnull().sum()

**We can see that there are no missing values in X_train and X_test.**

In [ ]:
# preview categorical variables in X_train

X_train[categorical].head()

In [ ]:
# import category encoders

import category_encoders as ce

**One Hot Encoding means that categorical variables are represented as binary.**

In [ ]:
# encode categorical variables with one-hot encoding

encoder = ce.OneHotEncoder(cols=['workclass', 'education', 'marital_status', 'occupation', 'relationship', 
                                 'race', 'sex', 'native_country'])

X_train = encoder.fit_transform(X_train)

X_test = encoder.transform(X_test)

In [ ]:
X_train.head()

In [ ]:
X_train.shape

**Similarly, I will take a look at the X_test set.**

In [ ]:
X_test.head()

In [ ]:
X_test.shape

In [ ]:
cols = X_train.columns
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)
X_train = pd.DataFrame(X_train, columns=[cols])
X_test = pd.DataFrame(X_test, columns=[cols])

**We now have X_train dataset ready to be fed into the Random Forest classifier.**

In [ ]:
# import Random Forest classifier

from sklearn.ensemble import RandomForestClassifier


# instantiate the classifier 

rfc = RandomForestClassifier(random_state=0)



# fit the model

rfc.fit(X_train, y_train)



# Predict the Test set results

y_pred = rfc.predict(X_test)

#Check accuracy score


from sklearn.metrics import accuracy_score

print('Model accuracy score with Random Forest (with 10 decision tree ) : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))



In [ ]:
# instantiate the classifier with n_estimators = 100

rfc_100 = RandomForestClassifier(n_estimators=100, random_state=0)



# fit the model to the training set

rfc_100.fit(X_train, y_train)



# Predict on the test set results

y_pred_100 = rfc_100.predict(X_test)



# Check accuracy score 

print('Model accuracy score with Random Forest (with 100 decision tree ) : {0:0.4f}'. format(accuracy_score(y_test, y_pred_100)))


**The expected accuracy increases with number of decision-trees in the model.**

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)
y_predict_dt = dt.predict(X_test)
acc_dt = metrics.accuracy_score(y_predict_dt,y_test)
print('The accuracy of the Decision Tree is', acc_dt)

In [ ]:
sv = SVC() #select the algorithm
sv.fit(X_train,y_train) # we train the algorithm with the training data and the training output
y_predict_svm = sv.predict(X_test) #now we pass the testing data to the trained algorithm
acc_svm = metrics.accuracy_score(y_predict_svm,y_test)
print('The accuracy of the SVM is:', acc_svm)

In [ ]:
# Print the Confusion Matrix and slice it into four pieces

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

print('Confusion matrix\n\n', cm)

In [ ]:
# visualize confusion matrix with seaborn heatmap

cm_matrix = pd.DataFrame(data=cm, columns=['Actual Positive:1', 'Actual Negative:0'], 
                                 index=['Predict Positive:1', 'Predict Negative:0'])

sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='YlGnBu')


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))